In [1]:
import gspread

In [2]:
gc = gspread.oauth(credentials_filename='./credentials.json')

In [3]:
sht2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xEqT4kghsaAZYTdN7SN9UHN31Ehx29OykbG1Mjpm-go/edit?gid=0#gid=0')

sht2

<Spreadsheet 'Copy of Torch Arrival Data' id:1xEqT4kghsaAZYTdN7SN9UHN31Ehx29OykbG1Mjpm-go>

In [4]:
worksheet = sht2.get_worksheet(0)
worksheet = sht2.worksheet("Shopee")
worksheet

<Worksheet 'Shopee' id:0>

In [5]:
sht2.worksheets()

[<Worksheet 'Shopee' id:0>,
 <Worksheet 'Tokopedia' id:715003248>,
 <Worksheet 'TokoMargonda' id:1960044452>]

In [6]:
import pandas as pd

worksheet = sht2.worksheet("Shopee")
shopee = pd.DataFrame(worksheet.get_all_records())
worksheet = sht2.worksheet("Tokopedia")
tokopedia = pd.DataFrame(worksheet.get_all_records())
worksheet = sht2.worksheet("TokoMargonda")
margonda = pd.DataFrame(worksheet.get_all_records())

date = tanggal tercatatnya arrival

total_arrival = jumlah customer yang berkunjung (halaman produk + kunjungan toko)

channel = Channel penjualan Torch (value : Shopee, Tokopedia, Toko Margonda)

In [7]:
display(shopee.head())
display(tokopedia.head())
display(margonda.head())

,date,arrival
0,2023-01-01,443
1,2023-01-02,899
2,2023-01-03,576
3,2023-01-04,267
4,2023-01-05,271


,date,arrival
0,2023-01-01,263
1,2023-01-02,907
2,2023-01-03,31
3,2023-01-04,992
4,2023-01-05,602


,date,arrival_toko,arrival_toko_shopee,arrival_toko_tokopedia
0,2023-01-01,25,229,92
1,2023-01-02,48,1,17
2,2023-01-03,68,493,46
3,2023-01-04,48,287,88
4,2023-01-05,62,232,40


In [8]:
margonda["total_arrival"] = margonda["arrival_toko"]+margonda["arrival_toko_shopee"]+margonda["arrival_toko_tokopedia"]
margonda.head()

,date,arrival_toko,arrival_toko_shopee,arrival_toko_tokopedia,total_arrival
0,2023-01-01,25,229,92,346
1,2023-01-02,48,1,17,66
2,2023-01-03,68,493,46,607
3,2023-01-04,48,287,88,423
4,2023-01-05,62,232,40,334


In [9]:
len(shopee)+len(tokopedia)+len(margonda)

270

In [10]:
combined_df = pd.concat(
    [
        shopee.rename(columns={"arrival":"total_arrival"}).assign(channel='Shopee'),
        tokopedia.rename(columns={"arrival":"total_arrival"}).assign(channel='Tokopedia'),
        margonda[["date","total_arrival"]].assign(channel='Toko Margonda')
    ], 
    ignore_index=True
)
combined_df

,date,total_arrival,channel
0,2023-01-01,443,Shopee
1,2023-01-02,899,Shopee
2,2023-01-03,576,Shopee
3,2023-01-04,267,Shopee
4,2023-01-05,271,Shopee
...,...,...,...
265,2023-03-27,827,Toko Margonda
266,2023-03-28,389,Toko Margonda
267,2023-03-29,135,Toko Margonda
268,2023-03-30,311,Toko Margonda


In [11]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import bs4
import pandas as pd

In [13]:
# path_ke_driver = r'.\chromedriver.exe'
# service = Service(executable_path=path_ke_driver)
# service=service

user_data_path = r"/mnt/c/Users/dioni/AppData/Local/Google/Chrome/User Data" 
profile_directory = "Profile 3" 

In [14]:
# import time
# import undetected_chromedriver as uc
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
# import random
# import os

# # URL target
# url = "https://shopee.co.id/search?keyword=back%20pack&page=0&sortBy=sales"

# # Tidak perlu Service, tidak perlu Options (dia otomatis cari profil)
# # Namun, kita bisa tambahkan path profil jika mau
# # --- SOLUSI 2: Arahkan path langsung ke folder 'Profile 3' ---
# base_path = r"C:\Users\dioni\AppData\Local\Google\Chrome\User Data"
# profile_folder = "Profile 3"

# # Buat path lengkap ke folder profil
# # Hasilnya: C:\Users\dioni\AppData\Local\Google\Chrome\User Data\Profile 3
# full_profile_path = os.path.join(base_path, profile_folder)
# print(f"Mencoba memuat profil dari: {full_profile_path}")

# try:
#     # --- 2. GUNAKAN uc.Chrome() ---
#     # Dia akan otomatis mencari chromedriver yang cocok
#     driver = uc.Chrome(
#         user_data_dir=full_profile_path,
#     )
#     # ------------------------------
    
#     time.sleep(2)

#     print(f"Mencoba membuka URL: {url}")
#     driver.get(url)
    
#     print("URL berhasil dibuka! Menunggu 15 detik...")
#     time.sleep(2)


#     search_bar = driver.find_element(By.CLASS_NAME, "shopee-searchbar-input__input")

#     search_bar.clear()

#     time.sleep(2)

#     search_term = "Back Pack"
#     for char in search_term:
#         search_bar.send_keys(char)
#         time.sleep(random.uniform(0.05, 0.3))

#     search_button = driver.find_element(By.CSS_SELECTOR, ".btn.shopee-searchbar__search-button")
#     search_button.click()

#     time.sleep(1)

#     print("Search successful!")


# except Exception as e:
#     print(f"Gagal menjalankan driver atau membuka URL.")
#     print(f"Error: {e}")

# finally:
#     if 'driver' in locals():
#         input("Tekan Enter untuk menutup browser...")
#         driver.quit()
#         print("Driver ditutup.")

In [15]:
# import time
# import undetected_chromedriver as uc
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
# import random
# import os

# # --- 1. SET UP PROFIL DAN URL ---
# url = "https://shopee.co.id"
# base_path = r"C:\Users\dioni\AppData\Local\Google\Chrome\User Data"
# profile_folder = "Profile 3"
# full_profile_path = os.path.join(base_path, profile_folder)

# # --- 2. SET UP OPTIONS ---
# options = uc.ChromeOptions()
# options.add_argument("--start-maximized") # <-- Buat jendela terlihat normal
# options.add_argument("--disable-infobars")
# options.add_argument("--disable-extensions") # Nonaktifkan ekstensi jika mengganggu
# options.add_argument('--no-first-run --no-service-autorun --password-store=basic')

# try:
#     print(f"Memulai driver dengan profil: {full_profile_path}")
#     driver = uc.Chrome(
#         user_data_dir=full_profile_path,
#         options=options,
#         headless=False, # Pastikan tidak headless
#         use_subprocess=True
#     )
    
#     print("Driver dimulai, menunggu 5 detik agar profil siap...")
#     time.sleep(5)
    
#     print(f"Mencoba membuka URL: {url}")
#     driver.get(url)
    
#     print("Menunggu 7 detik hingga halaman dimuat sepenuhnya...")
#     time.sleep(7)

#     # --- 3. SIMULASI INTERAKSI MANUSIA ---
    
#     # A. Temukan Elemen (Gunakan XPath/CSS yang stabil)
#     search_bar_selector = "//input[@class='shopee-searchbar-input__input']"
#     search_button_selector = "//button[contains(@class, 'shopee-searchbar__search-button')]"
    
#     search_bar = driver.find_element(By.XPATH, search_bar_selector)
#     search_button = driver.find_element(By.XPATH, search_button_selector)

#     # B. Gerakkan Mouse ke Search Bar (Tiru Manusia)
#     print("Mensimulasikan gerakan mouse ke search bar...")
#     actions = ActionChains(driver)
#     actions.move_to_element(search_bar).pause(random.uniform(0.5, 1.5)).click().perform()

#     # C. Ketik Seperti Manusia (dengan jeda acak)
#     print("Mensimulasikan pengetikan...")
#     search_term = "Back Pack"
#     for char in search_term:
#         search_bar.send_keys(char)
#         time.sleep(random.uniform(0.08, 0.25)) # Jeda antar ketukan

#     # D. Jeda sejenak (manusia berpikir)
#     time.sleep(random.uniform(1.0, 2.0))

#     # E. Gerakkan Mouse ke Tombol Cari dan Klik
#     print("Mensimulasikan gerakan mouse ke tombol cari...")
#     actions = ActionChains(driver)
#     actions.move_to_element(search_button).pause(random.uniform(0.5, 1.0)).click().perform()
    
#     print("Pencarian berhasil diklik! Menunggu hasil...")
#     time.sleep(15) # Tunggu hasil
    
#     print(f"URL Saat Ini: {driver.current_url}")
#     if "verify" in driver.current_url:
#         print("GAGAL: Masih dialihkan ke halaman verifikasi.")
#     else:
#         print("BERHASIL: Sepertinya kita ada di halaman hasil pencarian!")

# except Exception as e:
#     print(f"Gagal menjalankan driver atau membuka URL.")
#     print(f"Error: {e}")

# finally:
#     if 'driver' in locals():
#         input("Tekan Enter untuk menutup browser...")
#         driver.quit()
#         print("Driver ditutup.")

In [12]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import random
import os

base_path = "/root/chrome-uc-profile"
os.makedirs(base_path, exist_ok=True)
profile_folder="Profile 1"

# URL target
url = "https://shopee.co.id/search?keyword=back%20pack&page=0&sortBy=sales"

options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-extensions") 
options.add_argument("--disable-popup-blocking")
options.add_argument("--start-maximized")

try:
    # --- 2. GUNAKAN uc.Chrome() ---
    # Dia akan otomatis mencari chromedriver yang cocok
    driver = uc.Chrome(user_data_dir=base_path,
                       profile_directory=profile_folder,
                       use_subprocess=False,
                       options=options)
    # ------------------------------
    
    time.sleep(10)
    
    print(f"Mencoba membuka URL: {url}")
    driver.get(url)
    
    print("URL berhasil dibuka! Menunggu 15 detik...")
    time.sleep(2)

    print("Search successful!")


except Exception as e:
    print(f"Gagal menjalankan driver atau membuka URL.")
    print(f"Error: {e}")

finally:
    if 'driver' in locals():
        input("Tekan Enter untuk menutup browser...")
        driver.quit()
        print("Driver ditutup.")

Mencoba membuka URL: https://shopee.co.id/search?keyword=back%20pack&page=0&sortBy=sales
URL berhasil dibuka! Menunggu 15 detik...
Search successful!
Driver ditutup.


In [15]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import random
import os

# Import baru yang penting
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# URL Halaman 1
page=0
start_url = f"https://shopee.co.id/search?keyword=back%20pack&page={0+page}&sortBy=sales"

base_path = "/root/chrome-uc-profile"
os.makedirs(base_path, exist_ok=True)
profile_folder="Profile 1"

options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-extensions") 
options.add_argument("--disable-popup-blocking")
options.add_argument("--start-maximized")

try:
    driver = uc.Chrome(user_data_dir=base_path,
                       profile_directory=profile_folder,
                       use_subprocess=False,
                       options=options)
    
    actions = ActionChains(driver)
    wait = WebDriverWait(driver, 15) # Tentukan waktu tunggu maksimal 15 detik

    print(f"Mencoba membuka URL: {start_url}")
    driver.get(start_url)
    
    print("URL Halaman 1 berhasil dibuka! Menunggu 10 detik...")
    time.sleep(10) # Tunggu awal agar halaman 1 termuat penuh
    
    page_html = driver.page_source
    file_name = f"shopee_page_{1}.html"

    try:
        with open(file_name, "w", encoding="utf-8") as f:
            f.write(page_html)
            print(f"Berhasil menyimpan HTML ke: {file_name}")
    except Exception as e_save:
        print(f"Gagal menyimpan file HTML: {e_save}")

    # Kita akan coba pindah dari halaman 1 ke 5
    for i in range(1, 3): # Akan loop 4 kali (untuk klik ke hal 2, 3, 4, 5)
        
        print(f"\n--- Memproses Halaman {i+1} ---")
        
        page = page + i
        start_url = f"https://shopee.co.id/search?keyword=back%20pack&page={0+page}&sortBy=sales"

        driver.get(start_url)
        time.sleep(10)

        page_html = driver.page_source
        file_name = f"shopee_page_{i+1}.html"

        try:
            with open(file_name, "w", encoding="utf-8") as f:
                f.write(page_html)
                print(f"Berhasil menyimpan HTML ke: {file_name}")
        except Exception as e_save:
            print(f"Gagal menyimpan file HTML: {e_save}")


except Exception as e:
    print(f"Gagal menjalankan driver atau membuka URL.")
    print(f"Error: {e}")

finally:
    if 'driver' in locals():
        input("Tekan Enter untuk menutup browser...")
        driver.quit()
        print("Driver ditutup.")

Mencoba membuka URL: https://shopee.co.id/search?keyword=back%20pack&page=0&sortBy=sales
URL Halaman 1 berhasil dibuka! Menunggu 10 detik...
Berhasil menyimpan HTML ke: shopee_page_1.html

--- Memproses Halaman 2 ---
Berhasil menyimpan HTML ke: shopee_page_2.html

--- Memproses Halaman 3 ---
Berhasil menyimpan HTML ke: shopee_page_3.html
Driver ditutup.
